# Извлекаем топ-3 модели для каждого источника

In [2]:
import os
import re
import pandas as pd


In [3]:
def extract_models_table(iqtree_file):
    with open(iqtree_file, 'r') as f:
        content = f.read()

    # Найти начало таблицы по заголовку
    header_match = re.search(r"Model\s+LogL\s+AIC\s+w-AIC\s+AICc\s+w-AICc\s+BIC\s+w-BIC", content)
    if not header_match:
        return None

    # Получаем всё, что идет после заголовка
    table_content = content[header_match.end():]
    lines = [line.strip() for line in table_content.split('\n') if line.strip()]

    data = []
    for line in lines:
        # Разделить строку по 1 или более пробелам
        parts = re.split(r'\s{1,}', line)
        
        # Проверяем, что в строке хотя бы 11 столбцов (иначе — конец таблицы)
        if len(parts) < 11:
            break

        try:
            model = parts[0]
            logl = parts[1]
            aic = parts[2]
            aic_status = parts[3]
            w_aic = parts[4]
            aicc = parts[5]
            aicc_status = parts[6]
            w_aicc = parts[7]
            bic = parts[8]
            bic_status = parts[9]
            w_bic = parts[10]

            data.append({
                'Model': model,
                'LogL': float(logl),
                'AIC': float(aic),
                'w-AIC': float(w_aic),
                'AICc': float(aicc),
                'w-AICc': float(w_aicc),
                'BIC': float(bic),
                'w-BIC': float(w_bic)
            })
        except ValueError:
            # Строка не содержит чисел в нужных позициях — пропускаем
            continue

    return pd.DataFrame(data)


In [4]:
folder_path = "../data/processed/05_seq_for_tree/model_finder/"

In [5]:
models_df = pd.DataFrame()

In [6]:
for filename in os.listdir(folder_path):
    if filename.endswith(".iqtree"):
        filepath = os.path.join(folder_path, filename)
        
        # Применяем функцию к файлу
        df = extract_models_table(filepath)

        # Убедимся, что столбец BIC приведен к float (на всякий случай)
        df["BIC"] = pd.to_numeric(df["BIC"], errors="coerce")
        
        # Извлекаем 3 строки с наименьшим BIC
        top3 = df.nsmallest(3, "BIC").copy()

        # Добавляем столбец source с именем файла без расширения
        top3["source"] = os.path.splitext(filename)[0]

        # Добавляем строки в итоговый DataFrame
        models_df = pd.concat([models_df, top3], ignore_index=True)


In [7]:
models_df

,Model,LogL,AIC,w-AIC,AICc,w-AICc,BIC,w-BIC,source
0,Q.yeast+R6,-108399.289,217084.578,9.990000e-01,217089.558,9.990000e-01,218090.962,9.990000e-01,Ppl
1,Q.pfam+R6,-108406.724,217099.448,5.890000e-04,217104.428,5.890000e-04,218105.832,5.900000e-04,Ppl
2,Q.yeast+I+R6,-108405.520,217099.040,7.230000e-04,217104.090,6.980000e-04,218112.462,2.140000e-05,Ppl
3,Q.pfam+F+I+R7,-165516.589,331399.177,2.060000e-02,331407.360,2.250000e-02,332687.068,9.600000e-01,JetC_ECOR67_homologues
4,Q.pfam+F+I+R8,-165510.729,331391.457,9.790000e-01,331399.821,9.770000e-01,332693.423,4.000000e-02,JetC_ECOR67_homologues
5,LG+F+I+R7,-165664.027,331694.053,1.920000e-66,331702.236,2.100000e-66,332981.943,8.930000e-65,JetC_ECOR67_homologues
6,Q.yeast+R5,-65923.394,132088.788,1.000000e+00,132092.348,1.000000e+00,132940.344,1.000000e+00,ABC_ATPase_PARIS
7,Q.yeast+R4,-65940.467,132118.934,2.840000e-07,132122.377,3.020000e-07,132956.414,3.240000e-04,ABC_ATPase_PARIS
8,LG+R5,-66129.291,132500.581,3.800000e-90,132504.142,3.800000e-90,133352.137,3.800000e-90,ABC_ATPase_PARIS
9,Q.pfam+R9,-242822.744,486687.488,1.000000e+00,486756.409,1.000000e+00,490354.105,1.000000e+00,PDC-M01A


# Выбираем лучшую модель

In [8]:
# 1. Частота появления каждой модели в топ-3
model_counts = models_df['Model'].value_counts()
print("Частота моделей в топ-3 по BIC:")
print(model_counts)

Частота моделей в топ-3 по BIC:
Model
LG+R10           5
Q.pfam+R10       5
LG+I+R8          3
Q.yeast+I+R9     3
LG+I+R9          2
LG+G4            2
Q.yeast+I+R10    2
Q.pfam+I+R10     2
Q.pfam+R9        2
LG+I+R10         2
Q.yeast+R6       1
LG+R5            1
Q.yeast+R4       1
Q.pfam+R6        1
Q.yeast+I+R6     1
Q.pfam+F+I+R7    1
Q.pfam+F+I+R8    1
Q.insect+G4      1
Q.pfam+R8        1
Q.pfam+I+R8      1
Q.yeast+G4       1
LG+F+I+R7        1
Q.yeast+R5       1
LG+R7            1
LG+R8            1
LG+R6            1
Q.yeast+R10      1
VT+R10           1
Q.pfam+I+R9      1
Q.pfam+G4        1
LG+I+G4          1
LG+R9            1
Q.yeast+I+R8     1
Name: count, dtype: int64


In [9]:
mean_bic_per_model = models_df.groupby('Model')['BIC'].mean().sort_values()
print("\nСредний BIC по каждой модели:")
print(mean_bic_per_model)


Средний BIC по каждой модели:
Model
Q.yeast+G4       8.796647e+03
Q.insect+G4      8.848928e+03
LG+G4            1.201917e+04
Q.pfam+G4        1.518724e+04
LG+I+G4          1.518920e+04
LG+R7            1.308437e+05
LG+R8            1.308548e+05
LG+R6            1.308603e+05
Q.yeast+R5       1.329403e+05
Q.yeast+R4       1.329564e+05
LG+R5            1.333521e+05
Q.yeast+R6       2.180910e+05
Q.pfam+R6        2.181058e+05
Q.yeast+I+R6     2.181125e+05
Q.pfam+F+I+R7    3.326871e+05
Q.pfam+F+I+R8    3.326934e+05
LG+F+I+R7        3.329819e+05
Q.pfam+R9        4.607670e+05
Q.pfam+R8        4.904152e+05
Q.pfam+I+R8      4.904177e+05
LG+I+R9          5.819882e+05
Q.pfam+I+R9      6.055881e+05
LG+I+R8          6.171821e+05
LG+R9            6.358679e+05
Q.yeast+I+R9     6.617059e+05
Q.yeast+I+R8     8.258800e+05
Q.pfam+I+R10     9.127507e+05
Q.yeast+I+R10    9.953949e+05
Q.yeast+R10      1.472428e+06
Q.pfam+R10       1.907965e+06
LG+R10           1.914915e+06
LG+I+R10         2.186105e+06
VT+

In [10]:
# 3. Минимальный BIC по каждому источнику
min_bic_per_source = models_df.groupby('source')['BIC'].min()
print("\nМинимальный BIC по каждому файлу:")
print(min_bic_per_source)

# 4. Лучшая модель (с минимальным BIC) по каждому файлу
idx_min = models_df.groupby('source')['BIC'].idxmin()
best_models_per_source = models_df.loc[idx_min, ['source', 'Model', 'BIC']]
print("\nЛучшая модель по BIC для каждого файла:")
print(best_models_per_source)


Минимальный BIC по каждому файлу:
source
AAA_ATPases                      15175.975
ABC_ATPase_PARIS                132940.344
AbiL                           1472427.672
Ea59PtuA_homologues            3874664.027
Ea59PtuA_homologuesPDC-M01A     387471.662
GajA                           3149740.397
HEC-01A_ECOR46_homologues       130843.676
HECs                            605583.848
IbfA_homologues                 635867.906
JetC_ECOR67_homologues          332687.068
LmuB                            825879.972
NsnB_ECOR37_homologues          431174.209
Old_homologues                 1219086.075
PD-T4_4A_ECOR58_homologues      771703.796
PDC-M01A                        490354.105
Ppl                             218090.962
PrrC                              8796.647
Name: BIC, dtype: float64

Лучшая модель по BIC для каждого файла:
                         source          Model          BIC
42                  AAA_ATPases          LG+G4    15175.975
6              ABC_ATPase_PARIS     Q.ye

In [11]:
# Количество последовательностей по каждому источнику
sequence_counts = {
    "AAA_ATPases": 10,
    "ABC_ATPase_PARIS": 58,
    "AbiL": 997,
    "Ea59PtuA_homologues": 2228,
    "Ea59PtuA_homologuesPDC-M01A": 195,
    "GajA": 1619,
    "HEC-01A_ECOR46_homologues": 133,
    "HECs": 445,
    "IbfA_homologues": 426,
    "JetC_ECOR67_homologues": 77,
    "LmuB": 296,
    "NsnB_ECOR37_homologues": 406,
    "Old_homologues": 702,
    "PDC-M01A": 254,
    "PD-T4_4A_ECOR58_homologues": 525,
    "Ppl": 68,
    "PrrC": 5
}

# Добавим веса по количеству последовательностей
models_df["weight"] = models_df["source"].map(sequence_counts)

# Отбросим строки без соответствия в словаре (на всякий случай)
models_df = models_df.dropna(subset=["weight"])

# Группировка по модели: усреднённый взвешенный BIC
weighted_bic = models_df.groupby("Model").apply(
    lambda x: (x["BIC"] * x["weight"]).sum() / x["weight"].sum()
).sort_values()

# Вывод отсортированного списка моделей по взвешенному BIC
print("Взвешенный BIC по моделям (чем меньше, тем лучше):")
print(weighted_bic)

Взвешенный BIC по моделям (чем меньше, тем лучше):
Model
Q.yeast+G4       8.796647e+03
Q.insect+G4      8.848928e+03
LG+G4            1.307144e+04
Q.pfam+G4        1.518724e+04
LG+I+G4          1.518920e+04
LG+R7            1.308437e+05
LG+R8            1.308548e+05
LG+R6            1.308603e+05
Q.yeast+R5       1.329403e+05
Q.yeast+R4       1.329564e+05
LG+R5            1.333521e+05
Q.yeast+R6       2.180910e+05
Q.pfam+R6        2.181058e+05
Q.yeast+I+R6     2.181125e+05
Q.pfam+F+I+R7    3.326871e+05
Q.pfam+F+I+R8    3.326934e+05
LG+F+I+R7        3.329819e+05
Q.pfam+R9        4.539530e+05
Q.pfam+R8        4.904152e+05
Q.pfam+I+R8      4.904177e+05
Q.pfam+I+R9      6.055881e+05
LG+R9            6.358679e+05
LG+I+R8          6.449300e+05
LG+I+R9          6.703043e+05
Q.yeast+I+R9     7.137724e+05
Q.yeast+I+R8     8.258800e+05
Q.pfam+I+R10     9.815753e+05
Q.yeast+I+R10    1.027663e+06
Q.yeast+R10      1.472428e+06
LG+I+R10         2.567608e+06
Q.pfam+R10       2.749434e+06
LG+R10       

/tmp/ipykernel_50585/911602209.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_bic = models_df.groupby("Model").apply(
